In [ ]:
# Gene Activity Matrix Construction & Comparison

In [5]:
import os
import pandas as pd
import numpy as np
import anndata as ad
import scanpy as sc
import pyranges as pr
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
import seaborn as sns

In [6]:
# Load filtered RNA and ATAC AnnData objects
rna_adata = ad.read_h5ad("results/files/ag_rna_annotated_cleaned.h5ad")
atac_adata = ad.read_h5ad("results/files/ag_atac_matrix.h5ad")

# Load peak BED file with peak_id column (consistent with atac_adata.var_names)
bed_df = pd.read_csv("data/10k_PBMC_Multiome_nextgem_Chromium_X_atac_peaks.bed", sep="\t", comment="#", header=None)
bed_df.columns = ['Chromosome', 'Start', 'End']
bed_df['peak_id'] = [f"peak_{i}" for i in range(len(bed_df))]

# Prepare TSS ±2kb gene regions
gtf = pr.read_gtf("data/gencode.v43.annotation.gtf")
genes = gtf[gtf.Feature == "gene"]
genes = genes[['Chromosome', 'Start', 'End', 'Strand', 'gene_name']]
genes.Start -= 2000
genes.End += 2000

# Overlap peaks with gene regions
peaks_pr = pr.PyRanges(bed_df)
genes_pr = genes
overlap = peaks_pr.join(genes_pr, apply_strand_suffix=False)
peak_gene_df = overlap.df[['peak_id', 'gene_name']]



In [ ]:
# Construct Gene Activity Matrix
peak_id_to_idx = {peak: idx for idx, peak in enumerate(atac_adata.var_names)}
gene_to_peak_idx = {}
for row in peak_gene_df.itertuples(index=False):
    peak_id, gene = row.peak_id, row.gene_name
    idx = peak_id_to_idx.get(peak_id)
    if idx is not None:
        gene_to_peak_idx.setdefault(gene, []).append(idx)

num_cells = atac_adata.shape[0]
genes_list = list(gene_to_peak_idx.keys())
X_gene_activity = np.zeros((len(genes_list), num_cells))

for i, gene in enumerate(genes_list):
    peak_idxs = gene_to_peak_idx[gene]
    if peak_idxs:
        X_gene_activity[i, :] = atac_adata.X[:, peak_idxs].sum(axis=1).A1

# Create AnnData object for gene activity
gene_activity_adata = ad.AnnData(
    X=csr_matrix(X_gene_activity.T),
    obs=atac_adata.obs.copy(),
    var=pd.DataFrame(index=genes_list)
)

# Normalize and log1p
sc.pp.normalize_total(gene_activity_adata, target_sum=1e4)
sc.pp.log1p(gene_activity_adata)

# Match gene activity cells to RNA cells
gene_activity_adata = gene_activity_adata[rna_adata.obs_names]
assert all(gene_activity_adata.obs_names == rna_adata.obs_names)

# Copy cell types
gene_activity_adata.obs["cell_type"] = rna_adata.obs["cell_type"]

# Save gene activity matrix
gene_activity_adata.write("results/files/ag_gene_activity.h5ad")


In [8]:
# Violin plot for one gene
gene = "INPP4B"
fig = plt.figure(constrained_layout=True)
sc.pl.violin(rna_adata, keys=gene, groupby="cell_type", show=False)
plt.title(f"RNA: {gene}")
plt.xticks(rotation=90)
plt.savefig("results/plots/ag_INPP4B_rna.png", dpi=300)
plt.close()

fig = plt.figure(constrained_layout=True)
sc.pl.violin(gene_activity_adata, keys=gene, groupby="cell_type", show=False)
plt.title(f"ATAC Activity: {gene}")
plt.xticks(rotation=90)
plt.savefig("results/plots/ag_INPP4B_atac.png", dpi=300)
plt.close()



<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

In [9]:
# Combined Violin Plot for Marker Genes
marker_genes = ["INPP4B", "VCAN", "CCL5", "CST3", "BANK1", "GNLY"]
all_data = []

for gene in marker_genes:
    rna_vals = rna_adata[:, gene].X.toarray().flatten()
    atac_vals = gene_activity_adata[:, gene].X.toarray().flatten()

    rna_df = pd.DataFrame({"value": rna_vals, "cell_type": rna_adata.obs['cell_type'].values, "gene": gene, "modality": "RNA"})
    atac_df = pd.DataFrame({"value": atac_vals, "cell_type": gene_activity_adata.obs['cell_type'].values, "gene": gene, "modality": "ATAC"})

    all_data.extend([rna_df, atac_df])

combined_df = pd.concat(all_data)
sns.set(style="whitegrid")
for gene in marker_genes:
    plt.figure(figsize=(10, 5))
    sns.violinplot(
        data=combined_df[combined_df["gene"] == gene],
        x="cell_type", y="value", hue="modality",
        split=True, inner="quart", palette="Set2"
    )
    plt.xticks(rotation=45, ha="right")
    plt.title(f"{gene}: RNA Expression vs ATAC Activity")
    plt.ylabel("Signal")
    plt.xlabel("Cell Type")
    plt.tight_layout()
    plt.legend(title="Modality", loc="upper right")
    plt.savefig(f"results/plots/ag_{gene}_combined_violin.png")
    plt.close()

In [ ]:
# Heatmap for RNA and ATAC
summary = []
for gene in marker_genes:
    for cell_type in rna_adata.obs['cell_type'].unique():
        rna_mean = rna_adata[rna_adata.obs['cell_type'] == cell_type][:, gene].X.mean()
        atac_mean = gene_activity_adata[gene_activity_adata.obs['cell_type'] == cell_type][:, gene].X.mean()
        summary.append([gene, cell_type, rna_mean, atac_mean])

summary_df = pd.DataFrame(summary, columns=["gene", "cell_type", "RNA", "ATAC"])

# Heatmaps
plt.figure(figsize=(10, 6))
sns.heatmap(summary_df.pivot(index="gene", columns="cell_type", values="RNA"), cmap="Blues", annot=True, fmt=".2f")
plt.title("RNA Expression per Cell Type")
plt.tight_layout()
plt.savefig("results/plots/ag_rna_heatmap.png")
plt.show()

plt.figure(figsize=(10, 6))
sns.heatmap(summary_df.pivot(index="gene", columns="cell_type", values="ATAC"), cmap="Greens", annot=True, fmt=".2f")
plt.title("ATAC Gene Activity per Cell Type")
plt.tight_layout()
plt.savefig("results/plots/ag_atac_heatmap.png")
plt.show()